# **INTRODUCTIONS**

---

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **DECEASED_RIVM - NL**

---

## Tables

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231]'))
CREATE TABLE [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] (
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[DECEASED_ACTUAL] [int] NULL,
	[DECEASED_CUMULATIVE] [int] NULL,
	[VRCODE] [varchar](5) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[WEEK_START] [datetime] NULL,
	[DECEASED_7D_AVG] [decimal](16, 2) NULL,
	[WEEK_START_LAG] [datetime] NULL,
	[DECEASED_7D_AVG_LAG] [decimal](16, 2) NULL
);
GO


Stored procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231]

    INSERT INTO [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] (
        [ID],
        [DATE_OF_REPORT],
        [DATE_OF_REPORT_UNIX],
        [DECEASED_ACTUAL],
        [DECEASED_CUMULATIVE],
        [VRCODE],
        [DATE_LAST_INSERTED],
        [WEEK_START],
        [DECEASED_7D_AVG],
        [WEEK_START_LAG],
        [DECEASED_7D_AVG_LAG]
    )
    SELECT 
        [ID],
        [DATE_OF_REPORT],
        [DATE_OF_REPORT_UNIX],
        [DECEASED_ACTUAL],
        [DECEASED_CUMULATIVE],
        [VRCODE],
        [DATE_LAST_INSERTED],
        [WEEK_START],
        [DECEASED_7D_AVG],
        [WEEK_START_LAG],
        [DECEASED_7D_AVG_LAG]
    FROM [VWSDEST].[RIVM_DECEASED_DAILY_PER_REGION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_DECEASED_DAILY_PER_REGION])
END


## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_NATIONAL_ARCHIVED_20221231]
AS
--Retrieve the totals deceased on NL level by summing the region numbers.
WITH BASE_CTE AS (
        SELECT
        SUM(DECEASED_ACTUAL) AS DECEASED_ACTUAL
    ,   DATE_OF_REPORT_UNIX
    , DATE_LAST_INSERTED
    FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231])
    GROUP BY DATE_OF_REPORT_UNIX, DATE_LAST_INSERTED
)
,
--Determine the cumulative number as COVID_TOTAL
SECOND_CTE AS
(
    SELECT
        a.DECEASED_ACTUAL AS COVID_DAILY
    ,   SUM(DECEASED_ACTUAL) OVER(ORDER BY DATE_OF_REPORT_UNIX ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS COVID_TOTAL
    ,   DATE_OF_REPORT_UNIX AS DATE_UNIX
    ,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX

    -- 7d average
	,LAG ([DATE_OF_REPORT_UNIX] ,6) OVER (ORDER BY [DATE_OF_REPORT_UNIX] ASC) WEEK_START
	,LAG ([DATE_OF_REPORT_UNIX] ,7) OVER (ORDER BY [DATE_OF_REPORT_UNIX] ASC) WEEK_START_LAG
	,AVG(CAST([DECEASED_ACTUAL] AS FLOAT)) OVER (ORDER BY DATE_OF_REPORT_UNIX ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as DECEASED_7D_AVG
	,AVG(CAST([DECEASED_ACTUAL] AS FLOAT)) OVER (ORDER BY DATE_OF_REPORT_UNIX ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as DECEASED_7D_AVG_LAG
	
    FROM BASE_CTE a
)

SELECT
    COVID_DAILY
,   IIF(WEEK_START IS NULL, NULL, ROUND(DECEASED_7D_AVG,1)) AS COVID_DAILY_MOVING_AVERAGE
,   COVID_TOTAL
,   DATE_UNIX
,   DATE_OF_INSERTION_UNIX
FROM SECOND_CTE
WHERE dbo.CONVERT_UNIX_TO_DATETIME(DATE_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'deceased_rivm',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_DECEASED_DAILY_NATIONAL_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'deceased_rivm_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## difference/deceased_rivm__covid_daily

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_RIVM__DECEASED_DAILY_NL_ARCHIVED_20221231] AS


--For NL, the deceased total is determined as the sum of the safety region numbers
WITH DECEASED_CTE AS (
SELECT
    SUM(DECEASED_ACTUAL) AS DECEASED_ACTUAL
,   DATE_OF_REPORT_UNIX
,   DATE_OF_REPORT
, DATE_LAST_INSERTED
FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231])
AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GROUP BY DATE_OF_REPORT_UNIX,DATE_OF_REPORT, DATE_LAST_INSERTED
)
,
LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM DECEASED_CTE
),
BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (ORDER BY [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]), 1, NULL) OVER (ORDER BY DATE_LAST_INSERTED,[DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]) -
		LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]), 1, NULL) OVER (ORDER BY DATE_LAST_INSERTED,[DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
FROM DECEASED_CTE
WHERE DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM DECEASED_CTE)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCES_RIVM__DECEASED_DAILY_NL_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

# **DECEASED_RIVM - VR**

---

## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_REGIONAL_ARCHIVED_20221231]
AS
SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   DECEASED_7D_AVG AS COVID_DAILY_MOVING_AVERAGE
,   DECEASED_CUMULATIVE AS COVID_TOTAL
,   VRCODE
,   DATE_OF_REPORT_UNIX AS DATE_UNIX
,   DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231])
AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
 


GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'deceased_rivm',
@proto_name VARCHAR(50) = 'VR%',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_DECEASED_DAILY_REGIONAL_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'deceased_rivm_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(1024) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

## difference/deceased_rivm__covid_daily

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_RIVM__DECEASED_DAILY_VR_ARCHIVED_20221231] AS



WITH LAST_DATE_OF_REPORT AS (
	SELECT
		VRCODE,
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] WITH(NOLOCK)
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] WITH(NOLOCK))
	AND DATE_OF_REPORT < '2023-01-01' -- added as RIVM reports 9999 after this date
	GROUP BY VRCODE
),
BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (PARTITION BY VRCODE ORDER BY DATE_LAST_INSERTED,[DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	VRCODE,
	LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]), 1, NULL) OVER (PARTITION BY VRCODE ORDER BY DATE_LAST_INSERTED,[DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]) -
		LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_ACTUAL]), 1, NULL) OVER (PARTITION BY VRCODE ORDER BY DATE_LAST_INSERTED,[DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] WITH(NOLOCK)
WHERE DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_REGION_ARCHIVED_20221231] WITH(NOLOCK))
)
SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
      ,T1.VRCODE
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[VRCODE] = T2.[VRCODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
 


GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily',
@proto_name VARCHAR(50) = 'VR',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCES_RIVM__DECEASED_DAILY_VR_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

# **DECEASED_RIVM - GM**

---

## Tables

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231]'))
CREATE TABLE [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231] (
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[MUNICIPALITY_CODE] [varchar](10) NULL,
	[OLD_DATE_OF_REPORT] [datetime] NULL,
	[OLD_DATE_OF_REPORT_UNIX] [bigint] NULL,
	[OLD_VALUE] [int] NULL,
	[DIFFERENCE] [int] NULL,
	[DECEASED_ACTUAL] [int] NULL,
	[DECEASED_CUMULATIVE] [int] NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[WEEK_START] [datetime] NULL,
	[DECEASED_7D_AVG] [decimal](16, 2) NULL,
	[WEEK_START_LAG] [datetime] NULL,
	[DECEASED_7D_AVG_LAG] [decimal](16, 2) NULL
);
GO


Stored procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231]

    INSERT INTO [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231] (
        [ID],
        [DATE_OF_REPORT],
        [DATE_OF_REPORT_UNIX],
        [MUNICIPALITY_CODE],
        [OLD_DATE_OF_REPORT],
        [OLD_DATE_OF_REPORT_UNIX],
        [OLD_VALUE],
        [DIFFERENCE],
        [DECEASED_ACTUAL],
        [DECEASED_CUMULATIVE],
        [DATE_LAST_INSERTED],
        [WEEK_START],
        [DECEASED_7D_AVG],
        [WEEK_START_LAG],
        [DECEASED_7D_AVG_LAG]
    )
    SELECT 
        [ID],
        [DATE_OF_REPORT],
        [DATE_OF_REPORT_UNIX],
        [MUNICIPALITY_CODE],
        [OLD_DATE_OF_REPORT],
        [OLD_DATE_OF_REPORT_UNIX],
        [OLD_VALUE],
        [DIFFERENCE],
        [DECEASED_ACTUAL],
        [DECEASED_CUMULATIVE],
        [DATE_LAST_INSERTED],
        [WEEK_START],
        [DECEASED_7D_AVG],
        [WEEK_START_LAG],
        [DECEASED_7D_AVG_LAG]
    FROM [VWSDEST].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY])
END


## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231]
AS

SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   DECEASED_7D_AVG AS COVID_DAILY_MOVING_AVERAGE
,	DECEASED_CUMULATIVE AS COVID_TOTAL
,	MUNICIPALITY_CODE AS GMCODE
,   DATE_OF_REPORT_UNIX AS DATE_UNIX
,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231])
AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
 


GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'deceased_rivm',
@proto_name VARCHAR(50) = 'GM%',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'deceased_rivm_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(1024) = 'COVID_DAILY|COVID_DAILY_MOVING_AVERAGE|COVID_TOTAL|DATE_UNIX|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

## difference/deceased_rivm__covid_daily

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_RIVM__DECEASED_DAILY_GM_ARCHIVED_20221231] AS 

	SELECT DATE_OF_REPORT
		  ,DATE_OF_REPORT_UNIX											AS NEW_DATE_UNIX
		  ,OLD_DATE_OF_REPORT_UNIX										AS OLD_DATE_UNIX
		  ,MUNICIPALITY_CODE											AS GMCODE
		  ,OLD_VALUE													AS OLD_VALUE
		  ,[DIFFERENCE]													AS [DIFFERENCE]
	FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231]
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY_ARCHIVED_20221231])
	AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
 


GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily',
@proto_name VARCHAR(50) = 'GM',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCES_RIVM__DECEASED_DAILY_GM_ARCHIVED_20221231',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/deceased_rivm__covid_daily_archived_20221231',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO